In [1]:
import torch
import torch.nn as nn
from models.transformer_pytorch import TransformerPyTorch
from models.transformer import Transformer
from hyperparameters import hyperparameters

vocab_size = 32000
batch_size = 2
seq_len = 5

pytorch_model: nn.Module = TransformerPyTorch(
    vocab_size=vocab_size,
    d_model=hyperparameters.transformer.hidden_size,
    num_heads=hyperparameters.transformer.num_heads,
    d_ff=hyperparameters.transformer.encoder_ffn_embed_dim,
    num_encoder_layers=hyperparameters.transformer.num_hidden_layers,
    num_decoder_layers=hyperparameters.transformer.num_hidden_layers,
    dropout=hyperparameters.transformer.dropout,
    max_len=hyperparameters.transformer.max_len
)
own_model = Transformer(
    src_vocab_size=vocab_size,
    tgt_vocab_size=vocab_size,
    d_model=hyperparameters.transformer.hidden_size,
    num_heads=hyperparameters.transformer.num_heads,
    d_ff=hyperparameters.transformer.encoder_ffn_embed_dim,
    num_encoder_layers=hyperparameters.transformer.num_hidden_layers,
    num_decoder_layers=hyperparameters.transformer.num_hidden_layers,
    dropout=hyperparameters.transformer.dropout,
    max_len=hyperparameters.transformer.max_len
)
criterion = nn.CrossEntropyLoss(ignore_index=0, reduction="mean")

# Dummy data
src = torch.randint(1, vocab_size, (batch_size, seq_len))
tgt = torch.randint(1, vocab_size, (batch_size, seq_len))

# Ensure no zeros in the middle (just for clarity)
# but you can keep them if you want to test pad
decoder_in = tgt[:, :-1]
labels = tgt[:, 1:]

logits = pytorch_model(src, decoder_in)  # shape [B, T-1, vocab_size]
logits = logits.transpose(1, 2)  # shape [B, vocab_size, T-1]

loss = criterion(logits, labels)  # shape [B, T-1]
print("Dummy test loss =", loss.item())

# Own model
logits = own_model(src, decoder_in)  # shape [B, T-1, vocab_size]
logits = logits.transpose(1, 2)  # shape [B, vocab_size, T-1]

loss = criterion(logits, labels)  # shape [B, T-1]
print("Dummy test loss on own model =", loss.item())

Dummy test loss = 10.64527416229248
Dummy test loss on own model = 10.447006225585938


c:\Users\Sondr\TransformerUQ\venv\Lib\site-packages\torch\nn\functional.py:5849: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


In [4]:
# Find average sentence length in the dataset
merged_path = "local/data/training/bpe_train.de"
total_len = 0
num_lines = 0

with open(merged_path, "r", encoding="utf-8") as f:
    for line in f:
        total_len += len(line.split())
        num_lines += 1

avg_len = total_len / num_lines
print("Average sentence length in the dataset =", avg_len)

Average sentence length in the dataset = 30.32287386028867


In [5]:
import pickle
from vocab import Vocabulary


vocab = pickle.load(open("local/vocab_shared.pkl", "rb")) # type: ignore
print("Vocab size =", len(vocab))

Vocab size = 32181


In [1]:
import torch

x = torch.tensor(
    [
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]
    ]
)
print(x.view(-1))
print(x.view(3, 3))
x[:1]

tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])
tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])


tensor([[1, 2, 3]])

In [15]:
from models.transformer_model import TransformerModel
from hyperparameters import hyperparameters
from uq.generate_with_uq import _enable_test_time_dropout
from torch import nn

vocab_size = 32000

hyperparameters.transformer.transformer_implementation = "pytorch"

model = TransformerModel(
    vocab_size=vocab_size,
    d_model=hyperparameters.transformer.hidden_size,
    num_heads=hyperparameters.transformer.num_heads,
    d_ff=hyperparameters.transformer.encoder_ffn_embed_dim,
    num_encoder_layers=hyperparameters.transformer.num_hidden_layers,
    num_decoder_layers=hyperparameters.transformer.num_hidden_layers,
    dropout=hyperparameters.transformer.dropout,
    max_len=hyperparameters.transformer.max_len
)

model.eval()

def dropout_repr(self):
    return f"{self.__class__.__name__}(p={self.p}, training={self.training})"
nn.Dropout.__repr__ = dropout_repr


def enable_fast_test_time_dropout(model: TransformerModel) -> None:
    """
    Enable dropout for the final decoder layer in the transformer.
    """
    final_decoder_layer = model.transformer.decoder.layers[-1]
    for module in final_decoder_layer.modules():
        if isinstance(module, nn.Dropout):
            module.train()

enable_fast_test_time_dropout(model)


def print_model_with_mode(module, indent=0):
    prefix = ' ' * indent
    mode = 'train' if module.training else 'eval'
    print(f"{prefix}{module.__class__.__name__} (mode={mode})")
    for name, child in module.named_children():
        print_model_with_mode(child, indent + 4)

print_model_with_mode(model)
# model


TransformerModel (mode=eval)
    Embedding (mode=eval)
    Dropout (mode=eval)
    PositionalEncoding (mode=eval)
    Transformer (mode=eval)
        TransformerEncoder (mode=eval)
            ModuleList (mode=eval)
                TransformerEncoderLayer (mode=eval)
                    MultiheadAttention (mode=eval)
                        NonDynamicallyQuantizableLinear (mode=eval)
                    Linear (mode=eval)
                    Dropout (mode=eval)
                    Linear (mode=eval)
                    LayerNorm (mode=eval)
                    LayerNorm (mode=eval)
                    Dropout (mode=eval)
                    Dropout (mode=eval)
                TransformerEncoderLayer (mode=eval)
                    MultiheadAttention (mode=eval)
                        NonDynamicallyQuantizableLinear (mode=eval)
                    Linear (mode=eval)
                    Dropout (mode=eval)
                    Linear (mode=eval)
                    LayerNorm (mode=eval)


In [1]:
from uq.acquisition_func import BLEU_mean_output_batch,BLEUVariance,VR_mpnet_base_distance
import torch
# Dummy data
sentences = [["Hello world", "Goodbye world", "Hi globe", "Hi you are cool","Hello world"],
             ["dogs are cool", "cats are cool", "dogs are nice", "cats are cool","doggie"],
             [
                "The W514 village association is once again hosting this great exhibition.",
                "Once again, the local club W514 is setting out this large exhibition.",
                "The local district association W514 is once again holding this large exhibition.",
                "The local association W514 is once again setting out this large exhibition.",
                "The W514 local association is once again organizing this large exhibition."
             ],             
            ]
bv=BLEUVariance()
vr=VR_mpnet_base_distance()
print("BLEU_mean_output_batch",BLEU_mean_output_batch(sentences))
print("BLEUVariance",bv(sentences,torch.zeros(2)))
print("VR_mpnet_base_distance",vr(sentences,torch.zeros(2)))


c:\Users\Jonah\Documents\gitRepos\TransformerUQ\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BLEU_mean_output_batch ['Hi globe', 'dogs are cool', 'The local association W514 is once again setting out this large exhibition.']
BLEUVariance tensor([1.9722e-31, 1.9722e-31, 1.9722e-31])
VR_mpnet_base_distance tensor([1.8255, 1.5468, 0.6451])


In [6]:
import tiktoken
from gpt2project.data_processing.load_commongen import generate_input_text
from gpt2project.gpt2model import GPT
from gpt2project.gpt2_generate import generate_autoregressivly_gpt2
import torch

from gpt2project.utils.decode import decode_token_list

model = GPT.from_pretrained("gpt2")
model.eval()
prompt = generate_input_text(["car", "fast", "park"])
tokenizer = tiktoken.get_encoding("gpt2")


loading weights from pretrained gpt: gpt2


In [7]:
from gpt2project.search_methods_gpt import (
    AutoregressiveInferenceResults,
    _clean_inference_results,
    greedy_search_gpt,
)
import torch.nn as nn
from gpt2project.utils.decode import decode_token_id_batch
from gpt2project.utils.print_generated_text import print_generated_text_with_colors
from hyperparameters import hyperparameters
from uq.generate_with_uq import _enable_test_time_dropout

_enable_test_time_dropout(model)

tgt_tokens = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
results = [
    generate_autoregressivly_gpt2(
        model, tokenizer, tgt_tokens, greedy_search_gpt, break_on_newline=True
    )
    for _ in range(10)
]
results = [result.token_ids.squeeze(0).cpu() for result in results]
generated_text = decode_token_id_batch(results, tokenizer)
print_generated_text_with_colors(generated_text, prompt)

Task: Generate a meaningful sentence using the provided words.

Example 1:
Words: field, look, stand.
Sentence: The player stood in the field looking at the batter.

Example 2:
Words: climb, building, side.
Sentence: I climbed the side of the building.

Now try:
Words: car, fast, park.
Sentence:
 I climbed the side of the building.

 I climbed the side of the building.

 I climbed the side of the car.

 I was driving fast.

 The player was fast.

 I climbed the car fast.

 I climbed the side of the car.

 I parked the car.

 I'm going to park.

 I I climbed the car.



In [5]:
prompt

'Task: Generate a meaningful sentence using the provided words.\n\nExample 1:\nWords: field, look, stand.\nSentence: The player stood in the field looking at the batter.\n\nExample 2:\nWords: climb, building, side.\nSentence: I climbed the side of the building.\n\nNow try:\nWords: however, transmitter, arbitrary.\nSentence:'

In [65]:
import numpy as np
import wandb
import matplotlib.pyplot as plt

# Resume your existing run
wandb.init(
    project="GPT2Project",
    resume="never",
    group="evaluation",
    dir="test-wandb",
    job_type="evaluation",
)

assert wandb.run is not None
current_step_of_run = wandb.run.step
print(f"Current step of run: {current_step_of_run}")

# Create your data
y = list(range(11, 1, -1))
y = np.array(y) * 100
x = list(range(1, 11))

# Create a matplotlib plot
plt.figure(figsize=(10, 6))
plt.plot(x, y, marker="o", linestyle="-", color="blue")
plt.title("Retention Curve")
plt.xlabel("Number of Samples")
plt.ylabel("Evaluation Score")
plt.grid(True)

# Save the figure to a temporary file
plt_filename = "retention_curve.png"
plt.savefig(plt_filename)
plt.close()

# Create a table with benchmark name and image
table = wandb.Table(columns=["benchmark_name", "plot", "max_score", "min_score"])

# Add a row to the table with benchmark name and image
benchmark_name = "GPT2 Evaluation"
max_score = y.max()
min_score = y.min()
table.add_data(benchmark_name, wandb.Image(plt_filename), max_score, min_score)

# Log the table to W&B
wandb.log({"benchmark_results": table})

# Also add to summary to make it appear on the dashboard
# wandb.run.summary["benchmark_table"] = table

# Also save the image directly
# wandb.save(plt_filename)

wandb.finish()

Current step of run: 0


interactive(children=(SelectMultiple(description='Models', index=(0, 1, 2), options=('Model A', 'Model B', 'Mo…

Output()

In [7]:
from pathlib import Path
from typing import List, Literal, Tuple
from gpt2project.data_processing.commongen_dataset import CommonGen
from gpt2project.data_processing.lambada_dataset import Lambada
from gpt2project.data_processing.squad_dataset import Squad
import torch


def get_cached_generations(
    model: Literal["BayesformerGPT", "GPT"],
) -> List[str]:
    path = Path(
        f"local/gpt-evaluation-cache/full_run_24032025/Squad_{model}_mcdoTrue_greedy_search_gpt_TargetUsageEval.pt"
    )
    cached_data: Tuple[List[List[str]], torch.Tensor] = torch.load(path)
    generated_text = cached_data[0][1]  # BALD generations
    return generated_text


dataset = Squad()
dataset_examples = dataset.get_all_examples()

transformer_generated_text = get_cached_generations("GPT")
bayesformer_generated_text = get_cached_generations("BayesformerGPT")

assert len(transformer_generated_text) == len(dataset_examples)
assert len(bayesformer_generated_text) == len(dataset_examples)


def blue(text: str) -> str:
    return f"\033[94m{text}\033[0m"


def red(text: str) -> str:
    return f"\033[91m{text}\033[0m"

def green(text: str) -> str:
    return f"\033[92m{text}\033[0m"


full_prompt = dataset_examples[0].prompt
print(full_prompt)

for i, (dataset_example, transformer_generated_text, bayesformer_generated_text) in enumerate(zip(
    dataset_examples, transformer_generated_text, bayesformer_generated_text
)):
    print("\n##### Example " + str(i) + " #####")
    print(dataset_example.prompt + "\n" + green("Targets: " + str(dataset_example.targets)) + "\n" + blue("GPT: " + transformer_generated_text) + "\n" + red("BayesformerGPT: " + bayesformer_generated_text))

    if i > 200:
        break


Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
Question: Which NFL team represented the AFC at Super Bowl 50?
Answer:

##### Example 0 #####
Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for 

/tmp/ipykernel_2456079/2614843488.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cached_data: Tuple[List[List[str]], torch.Tensor] = torch.load(path)


In [21]:
dataset = CommonGen()


def number_of_concepts_in_prompt(prompt: str) -> int:
    return len(
        prompt.split("Words:")[-1]
        .split("Sentence:")[0]
        .strip()
        .split(",")
    )


sum(
    [number_of_concepts_in_prompt(example.prompt) for example in dataset_examples]
) / len(dataset_examples)

Rows in merged dataset: 993


3.7552870090634443